# Reprocessing data from MAGIC/PIC base

In [1]:
# We start with some code to access the table at http://vobs.magic.pic.es/fits/
def get_table_from_url(url):
    from bs4 import BeautifulSoup as BS
    import urllib2
    soup = BS(urllib2.urlopen(url).read())
    table = soup.find('table',{'class','mytable'})
    return table

url = 'http://vobs.magic.pic.es/fits/'
table_pic_html = get_table_from_url(url)

In [2]:
from IPython.display import HTML
HTML(unicode(table_pic_html))

In [3]:
# This table now is parsed to get the columns
def get_table_fields(table):

    def get_doi_url(url):
        # The url of the reference ("aurl").
        from bs4 import BeautifulSoup as BS
        import urllib2
        import re
        _ads = 'adsabs'
        _cds = 'cdsads'
        _axv = 'arxiv'
        if (_ads in url or _cds in url or _axv in url):
            soup = BS(urllib2.urlopen (url))
            trs = soup.findAll('tr')
            filter(lambda x: 'doi' in x.get_text().lower(), trs)
            tr = filter(lambda x: 'doi' in x.get_text().lower(), trs)[0]
            doi = tr.get_text()
            url = 'http://dx.doi.org/'
            url += re.sub('DOI:','',re.sub('\n','',doi))
        return url

    def process_row(row):
        cells = row.findAll('td')
        if len(cells)==5:
            # Object source name(s) (can be more then one comma separated)
            src = cells[0].find(text=True)
            src = src.strip()
            # Article reference (url), usually a ref to ads
            art = cells[1].find('a',href=True)
            aurl = art['href']
            durl = get_doi_url(aurl)
            # We skip year of publication (third column)
            # as well as bibcode reference (fourth column)
            #ref = cells[3].find(text=True).encode('utf8')
            # FITS file link for downloading it in the near future
            fits = cells[4]
            file = fits.find('a',href=True)
            try:
                file = file['href']
            except:
                file = None
            furl = url+file if file!=None else '_NULL_'
            return (src,durl,file)
        return None
    
    magic_table = {'SOURCE':[], 'DOI':[], 'FITS':[]}
    for row in table.findAll('tr'):
        vals = process_row(row)
        if vals is not None:
            src,durl,file = vals
            magic_table['SOURCE'].append(src)
            magic_table['DOI'].append(durl)
            magic_table['FITS'].append(file)
    return magic_table
    
table_pic_dict = get_table_fields(table_pic_html)

del table_pic_html
#tf = table_filtered
#for i in range(len(tf['source'])):
#    print("%s : %s : %s"%(tf['source'][i],tf['doi'][i],tf['fits'][i]))

In [4]:
def print_describe(table,include='all'):
    print table.describe(include=include)
    print "\n-> Has Nil?"
    hows_nil = table.isnull().any()
    print hows_nil
    for c in hows_nil.index:
        if not hows_nil[c]: continue
        print "\n-> Indexes where column '{}' is null:".format(c)
        print table[table[c].isnull()].index.values


In [5]:
import pandas as pd
table_pic = pd.DataFrame(table_pic_dict)

del table_pic_dict
print_describe(table_pic)

                                                   DOI  \
count                                               99   
unique                                              99   
top     http://dx.doi.org/10.1088/0004-637X/705/2/1624   
freq                                                 1   

                                       FITS         SOURCE  
count                                    67             99  
unique                                   67             68  
top     mfits/base/MAGIC_2010_CygnusX3.fits  Markarian 421  
freq                                      1              6  

-> Has Nil?
DOI       False
FITS       True
SOURCE    False
dtype: bool

-> Indexes where column 'FITS' is null:
[ 0  2  4  5  6  7 12 17 18 28 29 33 34 35 36 37 41 42 43 49 50 53 57 67 73
 83 84 88 90 93 96 98]


In [11]:
# Now we want to download the fits files
def download_fits(url,out):
    import wget
    filename = wget.download(url,out=out)
    return filename

def create_dir(dir):
    import os
    if not os.path.exists(dir):
        try:
            os.mkdir(dir)
        except:
            return False
    return True

def clean_dir(dir,ext="*"):
    import os
    from glob import glob
    create_dir(dir)
    if os.path.isdir(dir):
        files = glob(os.path.join(dir,ext))
        for f in files:
            os.remove(f)
        return True
    return False
    
def read_md5sum_file(md5txt='md5sum.txt'):
    md5_hashs,md5_files = [],[]
    with open(md5txt,'r') as mdf:
        for line in mdf.readline():
            _h,_f = line.split()
            md5_hashs.append(_h.strip())
            md5_files.append(_f.strip())
    md5 = dict(zip(md5_files,md5_hashs))
    return md5

def create_md5sum_file(files_list,md5txt='md5sum.txt',dir=None):
    import hashlib
    md5 = {}
    for f in files_list:
        fname = os.path.join(dir,f) if dir else f
        h = None
        with open(fname,'rb') as fp:
            h = hashlib.md5(fp.read()).hexdigest()
        md5.update({f:h})
    fname = os.path.join(dir,md5txt) if dir else md5txt
    with open(fname,'w') as fp:
        for file,hash in md5.iteritems():
            fp.write("%s    %s\n"%(hash,file))
    return md5

def read_dir_content(dir,ext='*.fits'):
    from glob import glob
    dir_files_list = glob(os.path.join(dir,ext))
    return [ os.path.basename(f) for f in dir_files_list ]

def is_exist_files(dir,files_list):
    import os
    # First we see if there is a file list (md5sum) to look for
    def check_md5sum(files_list,md5_file='md5sum.txt'):
        if os.path.isfile(md5_file):
            md5 = read_md5sum_file(md5_file)
            md5_files_list = md5.keys()
            leng_inters = len(set(md5_files_list).intersection(files_list))
            return leng_inters == len(files_list)
        # If there is *no* md5-file, return *None*
        return None
    # Also, check if the files are actually there (inside the dir)
    def check_glob(files_list,dir):
        files_ext = '*.fits'
        dir_files_list = read_dir_content(dir,files_ext)
        leng_matches = sum(map(lambda v: v in dir_files_list, files_list))
        return leng_matches == len(files_list)
    
    md5_file = 'md5sum.txt'
    md5_check = check_md5sum(files_list,md5_file)
    if md5_check in (True,False):
        return md5_check
    glob_check = check_glob(files_list,dir)
    if glob_check:
        create_md5sum_file(files_list,dir=dir)
    return glob_check


fits_download_dir = 'FITS_pic/'

FORCE_CLEAN = False
if FORCE_CLEAN:
    clean_dir(fits_download_dir)
else:
    create_dir(fits_download_dir)

import os
assert os.path.isdir(fits_download_dir),"FITS files dir do not exist!"

fits_files = table_pic.FITS.dropna().apply(os.path.basename)
if is_exist_files(fits_download_dir,fits_files):
    print("FITS files exist locally. Passing by download step..")
    fits_local = fits_files.apply(lambda f: os.path.join(fits_download_dir,f))
else:
    print("FITS files do not exist locally. Downloading them...")
    furls = url + table_pic.FITS.dropna()
    fits_local = furls.apply(lambda f: download_fits(f,fits_download_dir))
    md5s = create_md5sum_file(fits_files,dir=fits_download_dir)
    del furls

table_pic['FITS_local'] = fits_local

del fits_local,fits_files

print_describe(table_pic)

FITS files exist locally. Passing by download step..
                                                   DOI  \
count                                               99   
unique                                              99   
top     http://dx.doi.org/10.1088/0004-637X/705/2/1624   
freq                                                 1   

                                       FITS         SOURCE  \
count                                    67             99   
unique                                   67             68   
top     mfits/base/MAGIC_2010_CygnusX3.fits  Markarian 421   
freq                                      1              6   

                               FITS_local  
count                                  67  
unique                                 67  
top     FITS_pic/MAGIC_2010_CygnusX3.fits  
freq                                    1  

-> Has Nil?
DOI           False
FITS           True
SOURCE        False
FITS_local     True
dtype: bool

-> Indexes where co

#### Header keywords
 Primarily, regarding particularly MAGIC data, we should take a look at their [FITS format for MAGIC data](http://vobs.magic.pic.es/fits/mfits/tdas/tdas-fits.pdf) document, chapter 4. There we'll find the following list of keywords:

__FITS standard__, _highlighted by MAGIC_:
 * AUTHOR
 * DATE
 * DATE-OBS
 * EXTNAME
 * OBJECT
 * ORIGIN
 * TELESCOP
 
__MAGIC__, _to describe extension content_:
 * CONFLEVE
 * EFFICIEN
 * EMAX
 * EMIN
 * PERIOD
 * PHIMAX
 * PHIMIN
 * REFPAPER
 * SIZEMIN
 * SIZEMAX
 * TOBS
 * ZMIN
 * ZMAX
 * TZERO
 * VERSION

__MAGIC__, _SPECTRUM extension_:
 * ISINTEGR : 'F' means density flux; 'I' means integrated flux.
 
On the other hand, there is the [FITS standard](http://heasarc.gsfc.nasa.gov/docs/fcg/standard_dict.html) and the [FITS commons](http://heasarc.gsfc.nasa.gov/docs/fcg/common_dict.html) set of keywords.


```
This data dictionary lists the 53 keywords currently defined in the
FITS Standard:

(blank)  CROTAn   EQUINOX  NAXISn   TBCOLn   TUNITn
AUTHOR   CRPIXn   EXTEND   OBJECT   TDIMn    TZEROn
BITPIX   CRVALn   EXTLEVEL OBSERVER TDISPn   XTENSION
BLANK    CTYPEn   EXTNAME  ORIGIN   TELESCOP
BLOCKED  DATAMAX  EXTVER   PCOUNT   TFIELDS
BSCALE   DATAMIN  GCOUNT   PSCALn   TFORMn
BUNIT    DATE     GROUPS   PTYPEn   THEAP
BZERO    DATE-OBS HISTORY  PZEROn   TNULLn
CDELTn   END      INSTRUME REFERENC TSCALn
COMMENT  EPOCH    NAXIS    SIMPLE   TTYPEn
```

```
     Dictionary of Commonly Used FITS Keywords

This data dictionary contains FITS keywords that have been widely used
within the astronomical community.  It is recommended that these
keywords only be used as defined here.  These keywords may be grouped
within the following 7 broad categories:

1. Keywords that describe the data or the FITS file itself:

    TITLE FILENAME FILETYPE ROOTNAME
    PROGRAM CREATOR CONFIGUR
    NEXTEND HDUNAME HDUVER HDULEVEL
    TLMINn TLMAXn TDMINn TDMAXn TDBINn
    TSORTKEY PROGRAM CREATOR CONFIGUR
    HDUCLASS HDUDOC HDUVERS HDUCLASn

2.  Keywords that describe the observation:

    SUNANGLE MOONANGL
    RA DEC RA_NOM DEC_NOM
    RA_OBJ DEC_OBJ RA_PNT DEC_PNT PA_PNT
    RA_SCX DEC_SCX RA_SCY DEC_SXY RA_SCZ DEC_SCZ
    ORIENTAT AIRMASS LATITUDE
    OBJNAME OBS_ID

3.  Keywords that describe the instrument that took the data:

    OBS_MODE DATAMODE
    APERTURE DETNAM FILTER FILTERn GRATING GRATINGn
    SATURATE

4.  Keywords that give the date or duration of the observation:

    TIME-OBS TIME-END DATE-END
    EXPOSURE EXPTIME TELAPSE ELAPTIME ONTIME LIVETIME

5.  Keywords that denote non-standard FITS keyword format conventions:

    HIERARCH INHERIT CONTINUE

6.  File checksum keywords:

    CHECKSUM DATASUM CHECKVER

7.  Hierarchical file grouping keywords:

    GRPNAME GRPIDn GRPLCn
```

---

#### Improvement

 * Given all those lists of keywords, I will just add `RA` and `DEC`.


In [42]:
# The following keywords may not be seen in the Extended BinTableHDU header, but inside the PrimaryHDU' header.
# That's because the MAGIC data handling system decided to not repeat unnecessarily such data.
# See the 'General Keywords' section (4.1) of the document 'FITS Format for MAGIC data', version 0.2.
#
MAGIC_FITS_STANDARD_KEYWORDS = [
    'AUTHOR',
    'DATE',
    'DATE-OBS',
    'EXTNAME',
    'OBJECT',
    'ORIGIN',
    'TELESCOP']
MAGIC_FITS_EXTENSION_KEYWORDS = [
    'CONFLEVE',
    'EFFICIEN',
    'EMAX',
    'EMIN',
    'PERIOD',
    'PHIMAX',
    'PHIMIN',
    'REFPAPER',
    'SIZEMIN',
    'SIZEMAX',
    'TOBS',
    'ZMIN',
    'ZMAX',
    'TZERO',
    'VERSION']
# These keywords will be used to update the SPECTRUM extensions with *all* of them
FITS_KEYWORDS = MAGIC_FITS_STANDARD_KEYWORDS + MAGIC_FITS_EXTENSION_KEYWORDS

In [101]:
# Now we can process the fits files themselves.
# He start noting that we want the SPECTRUM Data Unit(s)
#  available (or not) in the fits files; discard the other DU.
# Things we want to do:
# - get the OBJECT name
# - get the each object position
# - get the observation date
# - transform the data vectors (x) to frequency(Hz) and (y) to flux(erg/s/cm2)
# Then we should follow the following workflow:
# - open the fits file
# - find the necessary data unit (SPECTRUM)
# - open its header
#  - get some keywords from the header
# - open its data; data here are vectors
#  - it can be from 2 to 4 vectors
#   - energy
#   - flux
#   - Denergy
#   - Dflux
#  - convert the ?energy vectors to 'Hz' units
#  - convert the ?flux vectors to 'erg/s/cm2' units

# Here we just define the functions we'll need..

def read_file(filename):
    from astropy.io import fits
    try:
        hdulist = fits.open(filename)
    except:
        hdulist = None
    return hdulist
    
def select_dataUnits(hdulist,du_name='SPECTRUM'):
    dui = [ i for i,du in enumerate(hdulist) if du_name in du.name ]
    dus = [ hdulist[ii] for ii in dui ]
    _du = filter(lambda x:du_name in x.name, hdulist)
    assert dus == _du
    return (dus,dui)
    
def read_header(spec,table):
    assert isinstance(table,dict)
    def read_keyword(header,table,word):
        table[word] = header.get(word, None)
    header = spec.header
    read_keyword(header,table, 'EXTNAME')
    read_keyword(header,table, 'OBJECT')
    read_keyword(header,table, 'DATE-OBS')
    read_keyword(header,table, 'SRCPOS1')
    read_keyword(header,table, 'SRCPOS2')

def attempt_fix_objectname(header):
    assert header['OBJECT'] is None
    import re
    _fn = header['FILENAME'][:-5]
    objname = re.sub('.*/MAGIC_20[0-9][0-9]_','',_fn)
    table['OBJECT'] = objname
    print '*** OBJECT name got from file name.'


def resolve_name(name):
    from astropy.coordinates import get_icrs_coordinates as get_coords
    try:
        icrs = get_coords(name)
        pos = (icrs.ra.value,icrs.dec.value)
    except:
        pos = None
    return pos

def read_data(spec,table):
    assert isinstance(table,dict)
    cols = zip(spec.columns.names,spec.columns.units)
    for n in ['energy','Denergy','flux','Dflux']:
        table[n+'_unit'] = None
        table[n] = None
    for n,u in cols:
        if (n == 'energy' or n == 'Denergy' or n == 'flux' or n == 'Dflux'):
            table[n+'_unit'] = u
            table[n] = spec.data[n]
        else:
            print "Different field:",n

def transf_data(table):
    from astropy import units
    Unit = units.Unit
    import numpy as np
    units.set_enabled_equivalencies(units.spectral())
    uEn = Unit('Hz')
    uFn = Unit('erg s-1 cm-2')
    uEc = Unit('TeV')
    conv = {Unit('ph TeV s-1 cm-2') : lambda x,y: (x/Unit('ph')).to(uFn),
            Unit('ph TeV-1 s-1 cm-2') : lambda x,y: ((y.to(uEc)**2)*(x/Unit('ph'))).to(uFn),
            Unit('ph s-1 cm-2') : lambda x,y: None,
            Unit('GeV') : lambda x: x.to(uEn, equivalencies=units.spectral())}

    vE = table['energy']
    uE = vE.unit
    vEn = conv[uE](vE)

    vF = table['flux']
    uF = vF.unit
    vFn = conv[uF](vF,vE)

    if vFn is None:
        print "vFn is None!?!:",str(uF)
        return False
    
    table['energy'] = vEn
    table['energy'].unit = vEn.unit
    table['flux'] = vFn
    table['flux'].unit = vFn.unit

    if 'Denergy' in table.colnames:
        vDE = table['Denergy']
        uDE = vDE.unit
        vDEn = conv[uDE](vDE)
        table['Denergy'] = vDEn
        table['Denergy'].unit = vDEn.unit
    else:
        uDE = Unit('')
        nullval = -999
        vDEn = np.asarray([nullval]*len(vE),dtype=int)
        table['Denergy'] = vDEn
        table['Denergy'].unit = uDE
        table['Denergy'].null = nullval

    if 'Dflux' in table.colnames:
        vDF = table['Dflux']
        uDF = vDF.unit
        vDFn = conv[uDF](vDF,vE) # Notice we use the energy bin/value of the measurement.
        table['Dflux'] = vDFn
        table['Dflux'].unit = vDFn.unit
    else:
        uDF = Unit('')
        nullval = -999
        vDFn = np.asarray([nullval]*len(vE),dtype=int)
        table['Dflux'] = vDFn
        table['Denergy'].unit = uDF
        table['Denergy'].null = nullval
    return True

def merge_header_keywords(header_p,header_s):
    # Extension's header has the highest priority; keywords there
    # should not be overwritten. Relevant keywords are the ones in:
    # 'FITS_KEYWORDS'
    f_header = {'COMMENT':[]}
    _kw = list(set(header_p.keys()).intersection(FITS_KEYWORDS))
    for k in _kw:
        f_header.update({k : header_p[k]})
    if 'COMMENT' in header_p.keys():
        f_header['COMMENT'].extend(header_p['COMMENT'])
    _kw = list(set(header_s.keys()).intersection(FITS_KEYWORDS))
    for k in _kw:
        f_header.update({k : header_s[k]})
    if 'COMMENT' in header_s.keys():
        f_header['COMMENT'].extend(header_s['COMMENT'])
    return f_header
    
def proc_fits_file(fn,source):
    out = []
    print "----------------------------------------------------------------------"
    print "\nProcessing file:",fn,

    hdulist = read_file(fn)
    if hdulist is None:
        print " ..failed: to open." 
        return None
    
    spectra,spec_du_i = select_dataUnits(hdulist)
    if not len(spec_du_i):
        print " ..failed: no SPECTRUM" 
        return None
    
    # First of all, let me see the contents of the first header
    phdu_header = hdulist[0].header
    print('PrimaryHDU header:')
    for key,value in phdu_header.items():
        print('{0} = {1}'.format(key, value))
    print("")

    for i,i_du in enumerate(spec_du_i):
        
        # Read the extension directly into an astropy.table
        from astropy.table import Table
        
        table = Table.read(fn,hdu=i_du)
        #print('Extension table metadata:')
        #for key, value in table.meta.items():
        #    print('{0} = {1}'.format(key, value))
        #print("")

        spec = hdulist[i_du]
        #print('Extension SPECTRUM header:')
        #for key,value in spec.header.items():
        #    print('{0} = {1}'.format(key, value))
        #print("")
        
        header_new = merge_header_keywords(phdu_header,table.meta)
        #print('New header dictionary:')
        #for key,value in header_new.items():
        #    print('{0} = {1}'.format(key, value))
        #print("")
        
        if not 'OBJECT' in header_new.keys():
            print("No 'OBJECT defined for this table. Passing by...")
            continue
            
        if ('SRCPOS1' in table.meta.keys()) and ('SRCPOS2' in table.meta.keys()):
            pos = table.meta['SRCPOS1'],table.meta['SRCPOS2']
        elif ('SRCPOS1' in phdu_header.keys()) and ('SRCPOS2' in phdu_header.keys()):
            pos = phdu_header['SRCPOS1'],phdu_header['SRCPOS2']
        else:
            pos = resolve_name(header_new['OBJECT'])
        if pos is None:
            pos = resolve_name(source)
            if pos is None:
                pos = (None,None)
            else:
                header_new['OBJECT'] = source
                print '*** NOP, No good either! Using PIC source.',source

        header_new['RA'],header_new['DEC'] = pos
        #for key,value in header_new.items():
        #    print('{0} = {1}'.format(key, value))
        #print("")
        
        table.meta.update(header_new)
    
        ret = transf_data(table)
        if ret is not True:
            continue
        out.append(table.copy())
    return out

#for i in range(len(table_pic)):
#    specs_list = proc_fits_file(table_pic.FITS_local[i],table_pic.SOURCE[i])
#assert False

In [110]:
table_pic['SPECTRUM'] = table_pic.apply(lambda x: proc_fits_file(x.FITS_local,x.SOURCE), axis=1)

----------------------------------------------------------------------

Processing file: nan  ..failed: to open.
----------------------------------------------------------------------

Processing file: FITS_pic/MAGIC_2015_CrabNebula.fits PrimaryHDU header:
SIMPLE = True
BITPIX = -32
NAXIS = 0
EXTEND = True
COMMENT =   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT =   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H
DATE = 2015-07-16T13:08:09
ORIGIN = MAGIC
AUTHOR = MAGIC Coll.
VERSION = 0.1
OBJECT = Crab Nebula
TELESCOP = MAGIC-Stereo
DATE-OBS = 2009-10
TOBS = 69.0
REFPAPER = 2014arXiv1406.6892M
COMMENT = This is a MAGIC fits file containing the results of the observations of
COMMENT = Crab Nebula -measurements over 3 decades; data from Oct 2009 to April 20
COMMENT = 11-.  Please consider it as VERY PRELIMINAR

Extension table metadata:
EXTNAME = SPECTRUM
ISINTEGR = F
DATE-OBS = 2009-10
TOBS = 69.0
EMIN = 50.0
EMAX = 30000.0
COMMENT

In [111]:

def fix_degeneracy(group):
    row = group.irow(0)
    columns = row.to_dict()
    tdf = {}
    tabs = row['SPECTRUM']
    tdf['SPECTRUM'] = tabs
    del columns['SPECTRUM']
    tdf['OBJECT'] = [ t.meta['OBJECT'] for t in tabs ]
    tdf['RA'] = [ t.meta['RA'] for t in tabs ]
    tdf['DEC'] = [ t.meta['DEC'] for t in tabs ]
    for c in columns: tdf[c] = [row[c]]*len(tabs)
    return pd.DataFrame(tdf)

table_proc = table_pic.dropna().groupby('DOI',group_keys=False).apply(fix_degeneracy).reset_index(drop=True)

/home/chbrandt/.conda/envs/magic/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: irow(i) is deprecated. Please use .iloc[i]
  app.launch_new_instance()


In [114]:
def dict_to_table(dict_tab):
    from astropy.table import Table,Column
    from astropy.units import Unit
    tab = dict_tab
    h_keys = ['filename','EXTNAME','OBJECT','SRCPOS1','SRCPOS2','DATE-OBS']
    t_cols = ['energy','Denergy','flux','Dflux']
    cols = []
    t = Table()
    for cname in t_cols:
        cdata = tab[cname]
        cunit = Unit(tab[cname+'_unit'])
        c = Column(data=cdata,name=cname,unit=cunit)
        cols.append(c)
    for c in cols:
        t.add_column(c)
    for kword in h_keys:
        t.meta[kword.upper()] = tab[kword]
    return t


#table_proc['SPECTRUM'] = table_proc.PROC.apply(dict_to_table)

def write_table(table,filename,odir):
    import os,re
    filename = os.path.basename(filename)
    filename = re.sub('[+]','p',filename)
    extname = '_' + table.meta['EXTNAME'] + '.fits'
    fitsfilename = os.path.join(odir,re.sub('.fits',extname,filename))
    table.write(fitsfilename,format='fits')
    votfilename = fitsfilename[:-5] + '.vot'
    table.write(votfilename,format='votable')
    return fitsfilename

outdir = 'FITS_out/'
clean_dir(outdir)

table_proc['FITS'] = table_proc.apply(lambda d:write_table(d.SPECTRUM,d.FITS_local,outdir),axis=1)

In [115]:
pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',10)
pd.set_option('display.width',100)
#del table_proc['PROC']
#del table_proc['SPECTRUM']

#print table_proc.describe(include='all')
print table_proc


           DEC                                                DOI  \
0    22.014500      http://dx.doi.org/10.1016/j.jheap.2015.01.002   
1    22.014500      http://dx.doi.org/10.1016/j.jheap.2015.01.002   
2    16.148200      http://dx.doi.org/10.1051/0004-6361/200811326   
3    16.148200      http://dx.doi.org/10.1051/0004-6361/200811326   
4    16.148200      http://dx.doi.org/10.1051/0004-6361/200811326   
5    11.190100      http://dx.doi.org/10.1051/0004-6361/200913607   
6    38.666700      http://dx.doi.org/10.1051/0004-6361/200913945   
7    38.666700      http://dx.doi.org/10.1051/0004-6361/200913945   
8    38.666700      http://dx.doi.org/10.1051/0004-6361/200913945   
9    38.666700      http://dx.doi.org/10.1051/0004-6361/200913945   
10   38.666700      http://dx.doi.org/10.1051/0004-6361/200913945   
11   38.666700      http://dx.doi.org/10.1051/0004-6361/200913945   
12   38.666700      http://dx.doi.org/10.1051/0004-6361/200913945   
13   38.666700      http://dx.doi.

/home/chbrandt/.conda/envs/magic/lib/python2.7/site-packages/pandas/core/format.py:2087: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e8).any()
/home/chbrandt/.conda/envs/magic/lib/python2.7/site-packages/pandas/core/format.py:2088: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) &
/home/chbrandt/.conda/envs/magic/lib/python2.7/site-packages/pandas/core/format.py:2089: RuntimeWarning: invalid value encountered in greater
  (abs_vals > 0)).any()


In [116]:
print_describe(table_proc,include=None)

              DEC          RA
count  129.000000  129.000000
mean    31.487762  164.916727
std     19.756908   97.078276
min    -30.225600    2.675560
25%     16.148200   83.633076
50%     35.201700  153.768000
75%     41.511695  216.751632
max     71.343400  356.765000

-> Has Nil?
DEC            True
DOI           False
FITS          False
FITS_local    False
OBJECT        False
PROC          False
RA             True
SOURCE        False
SPECTRUM      False
dtype: bool

-> Indexes where column 'DEC' is null:
[31]

-> Indexes where column 'RA' is null:
[31]


In [117]:
table_final = table_proc.dropna()[['OBJECT','RA','DEC','DOI','FITS']]

In [118]:
print_describe(table_final)

         OBJECT          RA         DEC                                              DOI  \
count       129  129.000000  129.000000                                              129   
unique       47         NaN         NaN                                               50   
top     Segue 1         NaN         NaN  http://dx.doi.org/10.1088/1475-7516/2014/02/008   
freq         14         NaN         NaN                                               14   
mean        NaN  164.916727   31.487762                                              NaN   
std         NaN   97.078276   19.756908                                              NaN   
min         NaN    2.675560  -30.225600                                              NaN   
25%         NaN   83.633076   16.148200                                              NaN   
50%         NaN  153.768000   35.201700                                              NaN   
75%         NaN  216.751632   41.511695                                         

In [119]:
print table_final

                     OBJECT          RA        DEC  \
0               Crab Nebula   83.633083  22.014500   
1               Crab Nebula   83.633083  22.014500   
2                   3C454.3   22.899400  16.148200   
3                   3C454.3   22.899400  16.148200   
4                   3C454.3   22.899400  16.148200   
5               PG 1553+113  238.929000  11.190100   
6             Markarian 421  165.250500  38.666700   
7             Markarian 421  165.250500  38.666700   
8             Markarian 421  165.250500  38.666700   
9             Markarian 421  165.250500  38.666700   
10            Markarian 421  165.250500  38.666700   
11            Markarian 421  165.250500  38.666700   
12            Markarian 421  165.250500  38.666700   
13            Markarian 421  165.250500  38.666700   
14            Markarian 421  165.250500  38.666700   
15                      M87  187.705500  12.391100   
16              B3 2247+381  342.520800  38.410280   
17              B3 2247+381 

In [120]:
table_final.to_csv('table_final_clean.csv')